This project was made in a team of 3 and I am thanking both of my team mates for this amazing experience I shared with them.

The dataset is available here: https://www.kaggle.com/usasha/million-music-playlists

And we took insipration for this code from a notebook on Kaggle.

# Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tqdm
import gensim
import pickle

# Open and read files

In [ ]:
meta = pd.read_csv('track_meta.tsv', sep='\t')
meta['band'] = meta['band'].astype(str)

id_to_band = meta.set_index('song_id')['band'].to_dict()

# Band recommandation

In [ ]:
for chunk in pd.read_csv('user_item_interaction.csv', 
                                   iterator=True,
                                   chunksize=1000000,
                                   lineterminator='\n',
                                   # limit the dataset lenght
                                   nrows=5000000,
                                   ##################################
                                  ):

    chunk.columns = ['user', 'track']
    chunk['band'] = chunk['track'].apply(lambda song_id: id_to_band.get(song_id, ''))
    chunk = chunk.groupby('user')['band'].apply(','.join).reset_index()

    chunk[['band']].to_csv('playlists.csv', header=None, index=None, mode='a')

In [ ]:
class TextToW2V(object):
    def __init__(self, file_path):
        self.file_path = file_path
    
    def __iter__(self):
        for line in open(self.file_path, 'r'):
            yield line.lower().split(',')[::-1]  # reverse order (make old -> new)

playlists = TextToW2V('playlists.csv')

In [ ]:
%%time
estimator = gensim.models.Word2Vec(playlists,
                                   window=15,
                                   min_count=30,
                                   sg=1,
                                   workers=4,
                                   ns_exponent=0.8,
                                  )

CPU times: user 54min 39s, sys: 3.79 s, total: 54min 42s
Wall time: 13min 48s


# Testing

### By using Band names':

In [ ]:
user_music = ['led zeppelin', 'Nirvana', 'Pink Floyd']
user_music = [m.lower().strip() for m in user_music]
predicted = estimator.predict_output_word(user_music)

[a[0] for a in predicted if a[0] not in user_music]

['alice cooper',
 'depeche mode - ultra - 1997',
 'iron maiden',
 'alice in chains',
 'acϟdc',
 'dio',
 'w.a.s.p.']

### By using song names' (to use if you replace in the code band by song to achieve music title recommandation):

In [ ]:
user_music = ['Chiquitita', 'Mr. Blue Sky', 'In the End', 'Hail to the King', 'High Hopes']
user_music = [m.lower().strip() for m in user_music]
predicted = estimator.predict_output_word(user_music)

[a[0] for a in predicted if a[0] not in user_music]

['hell to pay',
 'judgement day',
 'checkmate',
 'vodka',
 'the awakening',
 'so alive',
 'superhero',
 'falling apart',
 "don't cry"]

### By searching for a specific user in the dataset:

In [ ]:

user1 = chunk.song_name[(chunk['user'] == 40638)]
user1 = list(user1)[0]
user1 = user1.split(",")
user1

['Komsi_Komsa',
 '2000 год',
 'Не пилите',
 ' братцы',
 ' крейсер на иголки',
 'ВОЙСКА ДЯДИ ВАСИ(кавер)',
 'The Unforgiven',
 'Французский Поцелуй',
 'Жрать будет нечего',
 ' женюсь.',
 'Любит наш народ всякое гавно!',
 'Охуенный день',
 'Золотая клетка',
 'Гостья из будущего',
 'Крутые виражи',
 'В рок обработке',
 'Медлячек',
 'Ветеран',
 'Body Talk',
 'Anypla',
 'We Fight For Love',
 'Стюардесса по имени Жанна',
 'Война бывает детская',
 'Просвистела (Live in Essen)',
 'Feel this moment',
 'тайга прости',
 'Солдат',
 'Черный Бумер',
 'Малолетние',
 'Зелёная Шестёрка',
 'Евпатория',
 'Прошлась по коже иголка-иголочка',
 '',
 'Сколько лет',
 ' сколько зим',
 'Казачка',
 'За тебя',
 ' Родина-мать',
 'Я на рулетку',
 'Меня подцепили за оба крюка...',
 'Пора домой',
 'Был пацан и нет пацана',
 'Орел',
 'Ради тебя',
 'Таня',
 'Кепарик в клеточку',
 'Печальный парень',
 'Ярче люби',
 ' Ярче играй',
 'Постой душа',
 'Ты моя Самая любимая',
 '90-е как пуля',
 'Я на воздушном Шарике',
 'Голуб

In [ ]:
user_music = user1
user_music = [m.lower().strip() for m in user_music]
predicted = estimator.predict_output_word(user_music)

[a[0] for a in predicted if a[0] not in user_music]

['не плачь',
 'мама',
 'исповедь',
 'ах',
 'голуби',
 'люди',
 'журавли',
 'помнишь',
 'война',
 'ой']